In [1]:
from pathlib import Path
from typing import Tuple, Union, List, Dict, Iterable, Optional

import torch
import numpy as np
from tqdm.notebook import tqdm

from decomposer import Decomposer, DecomposerConfig
from recomposer import Recomposer, RecomposerConfig
from helpers import *

/home/webson/Research/congressional_adversary/congressional_env/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'decomposer.Decomposer' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


Vocab size = 138,442


In [2]:
# base_dir = Path('../../results/news/validation')
base_dir = Path('../../results/news/validation_transformer')
deno_space = load_decomposers_en_masse(base_dir, patterns='*/epoch2.pt')
# deno_space.update(load_decomposers_en_masse(base_dir, patterns='*/epoch3.pt'))

print(deno_space.keys())

Loading ../../results/news/validation_transformer/-3c/epoch2.pt
Loading ../../results/news/validation_transformer/debug cono only/epoch2.pt

dict_keys(['-3c/epoch2.pt', 'debug cono only/epoch2.pt'])


In [3]:
from evaluations.word_similarity.all_wordsim import mean_delta, read_plain_text, compare
import math

def convert(embed):
    word_vecs: Dict[str, np.array] = {}
    for word_id, vector in enumerate(embed):
        vector /= math.sqrt((vector ** 2).sum() + 1e-6)
        word_vecs[ITW[word_id]] = vector
    return word_vecs


def cf_WS(m1, m2):
    compare(convert(m1), convert(m2))

cf_WS(deno_space['-3c/epoch2.pt'], PE)


             Dataset             Rho       Reference      Delta >.02
    EN-MTurk-771.txt         49.32%         52.84%          -3.52%
  EN-RW-STANFORD.txt         44.11%         45.33%               𝜀
   EN-SIMLEX-999.txt         32.59%         32.92%               𝜀
    EN-MTurk-287.txt         47.04%         59.11%         -12.07%
       EN-YP-130.txt         47.69%         50.72%          -3.03%
   EN-WS-353-SIM.txt         66.36%         68.51%          -2.16%
    EN-MEN-TR-3k.txt         56.20%         52.49%           3.71%
   EN-WS-353-REL.txt         39.53%         44.70%          -5.17%
     EN-VERB-143.txt         20.39%         29.87%          -9.48%
   EN-WS-353-ALL.txt         52.24%         58.16%          -5.92%
 EN-SimVerb-3500.txt         25.62%         26.25%               𝜀
Mean Delta = -3.6195%


In [ ]:
def cf(query):
    GD(query)
    print('Pretrained:')
    nearest_neighbors(query, sup_PE)
    print('Our model:')
    nearest_neighbors(query, deno_space['-3c/epoch2.pt'])

In [ ]:
cf('estate_tax')
cf('death_tax')

In [ ]:
cf('undocumented_workers')
cf('illegal_aliens')
cf('chain_migration')

In [ ]:
cf('obamacare')
cf('aca')
cf('public_option')
cf('single_payer')
cf('socialized_medicine')

In [ ]:
cf('leftists')
cf('antifa')

In [ ]:
cf('guns')

In [ ]:
cf('trade')

In [ ]:
cf('crooked_hillary')
cf('crazy_bernie')
cf('lyin_ted')
cf('little_marco')

In [ ]:
cf('lgbt')

## Clustering

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

sns.set()

def plot(
        coordinates: np.ndarray,
        words: List[GroundedWord],
        path: Path
        ) -> None:
    fig, ax = plt.subplots(figsize=(15, 10))
#     skew = [w.R_ratio for w in words]
#     freq = [w.freq for w in words]
    sns.scatterplot(
        coordinates[:, 0], coordinates[:, 1],
#         hue=skew, palette='coolwarm',  # hue_norm=(0, 1),
#         size=freq, sizes=(200, 1000),
        legend=None, ax=ax)
    for coord, w in zip(coordinates, words):
        ax.annotate(w.word, coord, fontsize=20)
    with open(path, 'wb') as file:
        fig.savefig(file, dpi=300)
    plt.close(fig)


def plot_categorical(
        coordinates: np.ndarray,
        words: List[GroundedWord],
        path: Path,
        fancy: bool = True
        ) -> None:
    if fancy:
        fig, ax = plt.subplots(figsize=(20, 10))
        categories = [w.majority_deno for w in words]
        freq = [w.freq for w in words]
        sns.scatterplot(
            coordinates[:, 0], coordinates[:, 1],
            hue=categories, palette='muted', hue_norm=(0, 1),
            size=freq, sizes=(200, 1000),
            legend='brief', 
            ax=ax)
        chartBox = ax.get_position()
        ax.set_position(  # adjust legend
            [chartBox.x0, chartBox.y0, chartBox.width * 0.6, chartBox.height])
        ax.legend(loc='upper center', bbox_to_anchor=(1.45, 0.8), ncol=1)
    else:
        fig, ax = plt.subplots(figsize=(20, 10))
        freq = [w.freq for w in words]
        sns.scatterplot(
            coordinates[:, 0], coordinates[:, 1], ax=ax)

    for coord, w in zip(coordinates, words):
        ax.annotate(w.word, coord, fontsize=12)
    with open(path, 'wb') as file:
        fig.savefig(file, dpi=300)
    plt.close(fig)


def graph_en_masse(
        models: Dict[str, np.ndarray],
        out_dir: Path,
        reduction: str,  # 'PCA', 'TSNE', or 'both'
        words: List[GroundedWord],
        # hues: Union[List[float], List[int]],
        # sizes: List[int],
        perplexity: Optional[int] = None,
        categorical: bool = False
        ) -> None:
    Path.mkdir(out_dir, parents=True, exist_ok=True)
    word_ids = np.array([w.word_id for w in words])
    for model_name, embed in tqdm(models.items()):
        space = embed[word_ids]
        if reduction == 'PCA':
            visual = PCA(n_components=2).fit_transform(space)
        elif reduction == 'TSNE':
            assert perplexity is not None
            visual = TSNE(
                perplexity=perplexity, learning_rate=10,
                n_iter=5000, n_iter_without_progress=1000).fit_transform(space)
        elif reduction == 'both':
            assert perplexity is not None
            space = PCA(n_components=30).fit_transform(space)
            visual = TSNE(
                perplexity=perplexity, learning_rate=10,
                n_iter=5000, n_iter_without_progress=1000).fit_transform(space)
        else:
            raise ValueError('unknown dimension reduction method')
        if categorical:
            plot_categorical(visual, words, out_dir / f'{model_name}.png')
        else:
            plot(visual, words, out_dir / f'{model_name}.png')


In [ ]:
cherry_words = [
    'government', 'washington',
    'estate_tax', 'death_tax',
    'public_option', 'governmentrun',
    'foreign_trade', 'international_trade',
    'cut_taxes', 'trickledown'
]

cherry_words = [GroundedWord(w) for w in cherry_words]

In [ ]:
models = deno_space
stuff = polarization

graph_en_masse(
    models, out_dir=base_dir / 'cherry/topic/t-SNE p5',
    reduction='TSNE', perplexity=5, words=stuff, categorical=True)
graph_en_masse(
    models, out_dir=base_dir / 'cherry/topic/t-SNE p3',
    reduction='TSNE', perplexity=3, words=stuff, categorical=True)
graph_en_masse(
    models, out_dir=base_dir / 'cherry/topic/t-SNE p2',
    reduction='TSNE', perplexity=2, words=stuff, categorical=True)

In [ ]:
models = cono_space

graph_en_masse(
    models, out_dir=base_dir / 'cherry/party/t-SNE p5',
    reduction='TSNE', perplexity=5, words=cherry_words, categorical=False)

graph_en_masse(
    models, out_dir=base_dir / 'cherry/party/t-SNE p3',
    reduction='TSNE', perplexity=3, words=cherry_words, categorical=False)

graph_en_masse(
    models, out_dir=base_dir / 'cherry/party/t-SNE p2',
    reduction='TSNE', perplexity=2, words=cherry_words, categorical=False)

In [ ]:
models = deno_space

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed deno/party/t-SNE p25',
    reduction='TSNE', perplexity=25,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed deno/party/t-SNE p50',
    reduction='TSNE', perplexity=50,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

In [ ]:
models = cono_space

graph_en_masse(
    models, out_dir=f'{base_dir}/decomposed cono/topic/t-SNE p5',
    reduction='TSNE', perplexity=5,
    word_ids=J_ids, words=J_words, hues=J_deno, sizes=J_freq,
    categorical=True)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed cono/topic/t-SNE p3',
    reduction='TSNE', perplexity=3,
    word_ids=J_ids, words=J_words, hues=J_deno, sizes=J_freq,
    categorical=True)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed cono/topic/t-SNE p10',
    reduction='TSNE', perplexity=10,
    word_ids=J_ids, words=J_words, hues=J_deno, sizes=J_freq,
    categorical=True)

In [ ]:
models = cono_space

# graph_en_masse(
#     models,
#     out_dir=f'{base_dir}/Joint/topic/PCA',
#     reduction='PCA',
#     word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed cono/party/t-SNE p25',
    reduction='TSNE', perplexity=25,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

graph_en_masse(
    models,
    out_dir=f'{base_dir}/decomposed cono/party/t-SNE p50',
    reduction='TSNE', perplexity=50,
    word_ids=J_ids, words=J_words, hues=J_skew, sizes=J_freq)

# Homogeneity V-Measure

In [ ]:
# Deno space, eval deno, higher is better
for model_name, model in deno_space.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=True, top_k=10)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')
#     print(pred_labels)

In [ ]:
# Deno space, eval cono, lower is better
for model_name, model in deno_space.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=False, top_k=5)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')
#     print(pred_labels)

In [ ]:
# Cono space, eval cono, higher is better
for model_name, model in cono_space.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=False, top_k=5)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')

In [ ]:
# Cono space, eval deno, lower is better
for model_name, model in cono_space.items():
    cluster_labels, true_labels = NN_cluster_ids(
        model, J_ids, categorical=True, top_k=5)    
    homogeneity, completeness, v_measure = np.around(
        homogeneity_completeness_v_measure(true_labels, cluster_labels), 4)
    print(model_name, homogeneity, completeness, v_measure, sep='\t')